In [7]:
import networkx as nx
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
import json

# Creare un grafo di esempio
g = nx.erdos_renyi_graph(100, 0.1)

# Configurare il modello di diffusione
model = ep.SIRModel(g)
config = mc.Configuration()
config.add_model_parameter('beta', 0.1)  # tasso di trasmissione
config.add_model_parameter('gamma', 0.01)  # tasso di recupero
model.set_initial_status(config)

# Eseguire la simulazione
iterations = model.iteration_bunch(200)

# Convertire i dati delle iterazioni in formato JSON
diffusion_data = json.dumps(iterations)


In [9]:
from IPython.core.display import display, HTML

# Aggiungi i dati della diffusione come stringa
diffusion_data_str = diffusion_data

# HTML e JavaScript per la visualizzazione con Babylon.js
html_content = """
<!DOCTYPE html>
<html>
<head>
    <title>Graph Diffusion Visualization</title>
    <script src="https://cdn.babylonjs.com/babylon.js"></script>
    <script src="https://cdn.babylonjs.com/loaders/babylonjs.loaders.min.js"></script>
</head>
<body>
<canvas id="renderCanvas" style="width: 100%; height: 500px;"></canvas>

<script>
    var diffusionData = {diffusion_data};

    // Configura Babylon.js
    var canvas = document.getElementById('renderCanvas');
    var engine = new BABYLON.Engine(canvas, true);
    var createScene = function () {
        var scene = new BABYLON.Scene(engine);
        var camera = new BABYLON.ArcRotateCamera("camera", Math.PI / 2, Math.PI / 4, 10, BABYLON.Vector3.Zero(), scene);
        camera.attachControl(canvas, true);
        var light = new BABYLON.HemisphericLight("light", new BABYLON.Vector3(1, 1, 0), scene);

        // Crea i nodi
        var nodes = [];
        for (var i = 0; i < diffusionData[0].status.length; i++) {
            var sphere = BABYLON.MeshBuilder.CreateSphere("sphere" + i, {{diameter: 0.1}}, scene);
            sphere.position = new BABYLON.Vector3(Math.random() * 5 - 2.5, Math.random() * 5 - 2.5, Math.random() * 5 - 2.5);
            nodes.push(sphere);
        }

        // Anima la diffusione
        var iteration = 0;
        scene.registerBeforeRender(function () {
            if (iteration < diffusionData.length) {
                var status = diffusionData[iteration].status;
                for (var i = 0; i < status.length; i++) {
                    if (status[i] === 0) {
                        nodes[i].material = new BABYLON.StandardMaterial("infected", scene);
                        nodes[i].material.diffuseColor = new BABYLON.Color3(1, 0, 0);
                    } else if (status[i] === 1) {
                        nodes[i].material = new BABYLON.StandardMaterial("recovered", scene);
                        nodes[i].material.diffuseColor = new BABYLON.Color3(0, 1, 0);
                    } else {
                        nodes[i].material = new BABYLON.StandardMaterial("susceptible", scene);
                        nodes[i].material.diffuseColor = new BABYLON.Color3(0, 0, 1);
                    }
                }
                iteration++;
            }
        });

        return scene;
    };

    var scene = createScene();
    engine.runRenderLoop(function () {
        scene.render();
    });

    window.addEventListener('resize', function () {
        engine.resize();
    });
</script>
</body>
</html>
""".replace("{diffusion_data}", diffusion_data_str)

# Visualizza il contenuto HTML in una cella Jupyter
display(HTML(html_content))
